<a href="https://colab.research.google.com/github/Matceo/SyS./blob/main/Parcial3./S%26S%3APunto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Ejercicio1

!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

import os
import yt_dlp as youtube_dl
def download_ytvid_as_mp3(video_url, name):

    video_info=youtube_dl.YoutubeDL().extract_info(url=video_url,download=False)
    filename=f"{name}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])
    print("Download complete... {}".format(filename))

#Descargamos la canción de interés:

link_=["https://www.youtube.com/watch?v=4aPfwRbhqHs"]
name_=['Myke Towers']
for name, link in zip(name_, link_):
    print(name)
    download_ytvid_as_mp3(link, name)

#Para convertir a mp3:

import subprocess

for name in name_:
    subprocess.call(['ffmpeg', '-y', '-i', name+'.mp3',
                   name+'.wav'])
!pip install soundfile #Para instalar soundfile

import soundfile as sf #Para instalar pip install soundfile
from IPython.display import Audio #Librería para reproducir audios

#Para leer archivos .wav:

for name in name_:
    nombre_out="output.wav"
    x, fs=sf.read(name+'.wav')
    print('Frecuencia de muestreo %.2f[Hz]\naudio %s'%(fs, name+'.wav'))
xpro=x.copy() #Copiar los archivos para procesar
ti=45 #Tiempo de inicio a procesar en segundos
tf=50 #Tiempo final a procesar en segundos
m=xpro[int(ti*fs):int((tf*fs)), 0] #Señal de mensaje en el primer canal

In [ ]:
#Repoducir el fragmento original de la señal mensaje:

Audio(m, rate=fs)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

#Señal mensaje en tiempo y frecuencia:

tt = np.arange(ti,tf,1/fs) # vector de tiempo
plt.plot(tt,m) #graficar señal mensaje en el tiempo
plt.xlabel('Tiempo [s]')
plt.ylabel('m(t)')
plt.show()

#Fourier a la señal mensaje:

Mw=np.fft.rfft(m) #axis=0 permite aplicar FFT por cada columna de xpro
vf=np.fft.rfftfreq(m.shape[0], 1/fs) #Se crea el vector de frecuencias
plt.plot(vf, abs(Mw)) #Se grafica la magnitud
plt.title(r'Espectro de la Señal Mensaje')
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$|M[f]|$', fontsize=14)
plt.show()
plt.plot(vf, np.angle(Mw)) #Fase del espectro
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$<M[f]$', fontsize=14)
plt.show()
Fo=12000 #Frecuencia de la señal portadora
c=np.cos(2*np.pi*Fo*tt) #Señal portadora

In [ ]:
#Repoducimos la señal portadora:

Audio(c, rate=fs)

In [ ]:
#Señal portadora en el tiempo y la frecuencia:

plt.plot(tt, c) #Graficar la señal mensaje en el tiempo
plt.xlabel('$Tiempo [s]$')
plt.ylabel('$c(t)$')
plt.show()

#Fourier a la señal portadora:

Cw=np.fft.rfft(c) #axis=0 permite aplicar FFT por cada columna de xpro
plt.plot(vf, abs(Cw)) #Se grafica la magnitud
plt.title(r'Espectro de la Señal Portadora')
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$|C[f]|$', fontsize=14)
plt.show()
plt.plot(vf, np.angle(Cw)) #Fase del espectro
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$<C[f]$', fontsize=14)
plt.show()

#Para la señal modulada:

A1=2
y=A1*m*c #Señal modulada en AM

In [ ]:
#Repoducir la señal modulada:

Audio(y, rate=fs)

In [ ]:
plt.plot(tt, y) #Graficar la señal modulada en el tiempo
plt.xlabel('$Tiempo [s]$')
plt.ylabel('$y(t)$')
plt.show()

#Fourier a la señal modulada:

Yw=np.fft.rfft(y) #axis=0 permite aplicar FFT por cada columna de xpro
plt.plot(vf, abs(Yw)) #Se grafica la magnitud
plt.title(r'Espectro de la Señal Modulada')
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$|Y[f]|$', fontsize=14)
plt.show()
plt.plot(vf, np.angle(Yw)) #Fase del espectro
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$<Y[f]$', fontsize=14)
plt.show()

In [ ]:
#Etapa 1: Mixer:

d=y*c #Señal demodulada
Audio(d, rate=fs) #Repoducir la señal demodulada

In [ ]:
plt.plot(tt, d) #Graficar la señal demodulada en el tiempo
plt.xlabel('$Tiempo [s]$')
plt.ylabel('$d(t)$')
plt.show()

#Fourier a la señal demodulada:

Dw=np.fft.rfft(d) #axis=0 permite aplicar FFT por cada columna de xpro
plt.plot(vf, abs(Dw)) #Se grafica la magnitud
plt.title(r'Espectro de la Señal Modulada')
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$|D[f]|$', fontsize=14)
plt.show()
plt.plot(vf, np.angle(Dw)) #Fase del espectro
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$<D[f]$', fontsize=14)
plt.show()

#Etapa 2: Filtro Pasabajas:

from scipy.signal import butter, cheby1, cheby2, bessel, lfilter, freqz, freqz_zpk, filtfilt
import matplotlib.pyplot as plt
import numpy as np

#Diseño del filtro Butteworth:

N=10 #Orden del filtro
Wn=15000 #Banda pasante
filt='lowpass' #Tipo del filtro (pasa-baja, pasa-alta, pasabanda, rechazabanda)
out='zpk' #La funcion retorna los polos y ceros junto con la ganancia. Para usar numerador y denominador usar 'ba' junto con freqz
zeros, poles, gain = butter(N, Wn, btype=filt, output=out, fs=fs)
w, h=freqz_zpk(zeros, poles, gain, fs=fs) #Para indicar la longitud usar worN=K

from matplotlib.patches import Circle

#Gráfica de polos y ceros:

def plot_freq_response(filter_name, w, h, N): #Mostrar la respuesta en frecuencia del filtro
  fig=plt.figure()
  ax1=fig.add_subplot(1, 1, 1)
  ax1.set_title(filter_name +' Digital filter frequency response, order= '+str(N))
  ax1.plot(w, 20*np.log10(abs(h)), 'b')
  ax1.set_ylabel('Amplitud [dB]', color='b')
  ax1.set_xlabel('Frecuencia [Hz]')
  ax1.grid()
  ax2=ax1.twinx()
  angles=np.unwrap(np.angle(h))
  ax2.plot(w, angles, 'g')
  ax2.set_ylabel('Ángulo [radianes]', color='g')
  plt.axis('tight')
  plt.show()

def show_zp(z, p, title='Z-plane'): #Mostrar la ubicación de los polos y los ceros

  ax=plt.gca()
  ax.plot(np.real(z), np.imag(z), 'bo', fillstyle='none', ms=10)
  ax.plot(np.real(p), np.imag(p), 'rx', fillstyle='none', ms=10)
  unit_circle=Circle((0, 0), radius=1, fill=False,
                        color='black', ls='solid', alpha=0.9)
  ax.add_patch(unit_circle)
  ax.axvline(0, color='0.7')
  ax.axhline(0, color='0.7')
  plt.title(title)
  plt.xlabel(r'Re{$z$}')
  plt.ylabel(r'Im{$z$}')
  plt.axis('equal')
  plt.xlim((-2, 2))
  plt.ylim((-2, 2))
  plt.grid()

show_zp(zeros, poles, title='Polos y Ceros') #Gráfica de polos y ceros
plot_freq_response('Butterworth', w, h, N) #Gráfica del Diagrama de Bode del filtro

In [ ]:
#Aplicación del Filtro de Butterworth a la canción:

out='ba'
num, den=butter(N, Wn, btype=filt, output=out, fs=fs)
xf=lfilter(num, den, d, axis=0)
Audio(xf, rate=fs) #Audio con ruido filtrado

In [ ]:
#Etapa 3: Escalamiento de Amplitud:

me=(2/A1)*xf #Señal reescalada
Audio(me, rate=fs) #Audio con ruido filtrado

In [ ]:
plt.plot(tt, me) #Graficar señal recuperada
plt.xlabel('$Tiempo [s]$')
plt.ylabel('$m_e(t)$')
plt.show()

#Fourier a la señal recuperada:

Mew=np.fft.rfft(me) #axis=0 permite aplicar FFT por cada columna de xpro
plt.plot(vf, abs(Mew)) #Se grafica la magnitud
plt.title(r'Espectro de la Señal Recuperada')
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$|M_e[f]|$',fontsize=14)
plt.show()
plt.plot(vf, np.angle(Mew)) #Fase del espectro
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$<M_e[f]$', fontsize=14)
plt.show()

#Diseño del Filtro de Bessel:

N=4
out='ba'
filt='lowpass' #Tipo del filtro (pasa-baja, pasa-alta, pasabanda, rechazabanda)
normalization='mag' #Ajusta la frecuencia crítica según la respuesta que se desea
num, den=bessel(N, Wn=Wn, btype=filt, norm=normalization, output=out, fs=fs) #Regresa el numerador y el denominador. Si se quiere obtener los polos y ceros usar scipy.signal.tf2zpk
w, h=freqz(num, den, fs=fs)
plot_freq_response('Bessel', w, h, N)

In [ ]:
#Aplicación del Filtro de Bessel a la canción:

xf=lfilter(num, den, d, axis=0)
Audio(xf,rate=fs) #Audio con ruido filtrado

In [ ]:
#Escalamiento de la amplitud:

me=(2/A1)*xf #señal reescalada
Audio(me, rate=fs) #Audio con ruido filtrado

In [ ]:
plt.plot(tt, me) #Graficar la señal recuperada
plt.xlabel('$Tiempo [s]$')
plt.ylabel('$m_e(t)$')
plt.show()

#Diseño del Filtro de Chebyshev1:

N=4
ripple=10
out='zpk'
zeros, poles, gain=cheby1(N, rp=ripple, Wn=Wn, btype=filt, output=out, fs=fs)
w, h=freqz_zpk(zeros, poles, gain, fs=fs)
plot_freq_response('Chebyshev1', w, h, N)

In [ ]:
#Aplicación del Filtro de Chebyshev 1:

out='ba'
num, de=cheby1(N, rp=ripple, Wn=Wn, btype=filt, output=out, fs=fs)
xf=lfilter(num, den, d, axis=0)
Audio(xf, rate=fs) #Audio con ruido filtrado

In [ ]:
#Escalamiento de la amplitud:

me=(2/A1)*xf #Señal reescalada
Audio(me, rate=fs) #Audio con ruido filtrado

In [ ]:
plt.plot(tt, me) #Graficar señal recuperada
plt.xlabel('$Tiempo [s]$')
plt.ylabel('$m_e(t)$')
plt.show()

#Fourier señal recuperada:

Mew=np.fft.rfft(me) #axis=0 permite aplicar FFT por cada columna de xpro
plt.plot(vf, abs(Mew)) #Se grafica la magnitud
plt.title(r'Espectro de la Señal Recuperada')
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$|M_e[f]|$', fontsize=14)
plt.show()
plt.plot(vf, np.angle(Mew)) #Fase del espectro
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$<M_e[f]$', fontsize=14)
plt.show()

#Diseño del Filtro de Chebyshev 2:

N=4
ripple=20
out='ba' #No regresa los polos y ceros, sino la funcion de transferencia
num, den=cheby2(N, rs=ripple, Wn=Wn, btype=filt, output=out, fs=fs) #Regresa numerador y denominador. Si se quiere obtener los polos y ceros usar scipy.signal.tf2zpk
w, h=freqz(num, den, fs=fs)
plot_freq_response('Chebyshev2', w, h, N)

In [ ]:
#Aplicación del Filtro de Chebyshev 2:

xf=lfilter(num, den, d, axis=0)
Audio(xf, rate=fs) #Audio con ruido filtrado

In [ ]:
#Escalamiento de la amplitud:

me=(2/A1)*xf #Señal reescalada
Audio(me, rate=fs) #Audio con ruido filtrado

In [ ]:
plt.plot(tt, me) #Graficar señal recuperada
plt.xlabel('$Tiempo [s]$')
plt.ylabel('$m_e(t)$')
plt.show()

#Fourier a la señal recuperada:

Mew=np.fft.rfft(me) #axis=0 permite aplicar FFT por cada columna de xpro
plt.plot(vf, abs(Mew)) #Se grafica la magnitud
plt.title(r'Espectro de la Señal Recuperada')
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$|M_e[f]|$', fontsize=14)
plt.show()
plt.plot(vf, np.angle(Mew)) #Fase del espectro
plt.xlabel(r'$f[Hz]$', fontsize=14)
plt.ylabel(r'$<M_e[f]$', fontsize=14)
plt.show()